
## 20200619_00
efficient net のバグがあるのか確認するために resnet で実験

In [ ]:
import chainer
from chainer.dataset import convert
from chainercv.links import ResNet50

import melanoma

title = "20200619_00"

device = 0
batch_size = 16
width = 336
height = 224
epoch = 10
n_classes = 2
output_dir = melanoma.constants.PROJECT_ROOT / "results" / title
transfered_weights = None

model = ResNet50(n_class=2)
loss_func = melanoma.models.loss.get_sigmoid_loss_func(model)
optimizer = chainer.optimizers.Adam()
predictor = melanoma.predictor.Predictor(model, (width, height))

if device >= 0:
    model.to_gpu(device)

optimizer.setup(model)

train_ds, val_ds, test_ds = melanoma.dataset.DatasetBuilder(img_size=(width, height), n_classes=n_classes).build()

train_itr = chainer.iterators.MultiprocessIterator(train_ds, batch_size, repeat=True)
val_itr = chainer.iterators.MultiprocessIterator(val_ds, batch_size, repeat=False)
test_itr = chainer.iterators.MultiprocessIterator(test_ds, batch_size, repeat=False)

updater = chainer.training.StandardUpdater(
    train_itr, optimizer, converter=convert.concat_examples, loss_func=loss_func, device=device
)
evaluator = chainer.training.extensions.Evaluator(
    val_itr, model, device=device, eval_func=loss_func
)
trainer = melanoma.trainer.TrainerBuilder(updater, epoch ,evaluator, output_dir).build()


In [ ]:
# run train
import cProfile

profiler = cProfile.Profile()
melanoma.utility.start_pdb(
    lambda: profiler.runcall(trainer.run)
)

In [ ]:
# profiling
profiler.print_stats("cumulative")
profiler.dump_stats(melanoma.constants.RESULT_DIR / title / "profile.stat")

In [ ]:
# evaluation
chainer.serializers.load_npz(melanoma.constants.RESULT_DIR / title / "snapshot_model_{}.npz".format(epoch), model)
if device >= 0:
    model.to_gpu()
    predictor.to_gpu()
test_itr.reset()
melanoma.evaluate.evaluate(predictor, 
                           test_itr,
                           [l.name for l in melanoma.constants.Labels],
                           output_dir / f"{title}_eval",
                           device=device,
                           )

In [ ]:
# submission
npz_footer = epoch
chainer.serializers.load_npz(melanoma.constants.RESULT_DIR / title / "snapshot_model_{}.npz".format(npz_footer), model)
if device >= 0:
    model.to_gpu()
    predictor.to_gpu()

ds = melanoma.dataset.SubmissionDataset(melanoma.dataset.DATASET_ROOT / "test.csv")
itr = chainer.iterators.MultiprocessIterator(ds, batch_size, repeat=False)
melanoma.utility.start_pdb(
lambda: melanoma.evaluate.evaluate_submission(predictor, 
                           itr,
                           output_dir / f"{title}_submission_{npz_footer}",
                           device=device,
                          )
)
